<a href="https://colab.research.google.com/github/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetworks/NerualArtGeneration/Art_Generation_with_Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning & Art: Neural Style Transfer

## 1. Packages


In [3]:
# Importing relevant packages
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
import matplotlib.pyplot as imshow
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
import pprint
%matplotlib inline

# 2. Problem Statement

Nerual Style Transfer (NST) is one of the most fun and interesting optimzation techniques in deep learing. It merges two images, namely a **"content" image (C)** and **"style" image (S)**, to create a **"generated" image (G) **. The generated image G combines the "content" of the image C with the "style" of image S.

In this assignment, we are going to cobine the Louvre museum in Paris (content image, C) with the impressionist style of Claude Monet (content image S) to generate the following image:
